In [ ]:
import sys
sys.path.append('..')

In [2]:
import planetengine
import modelscripts

In [3]:
initial = {
    'temperatureField': {'IC': planetengine.initials.sinusoidal.IC()},
    'materialVar': {'IC': planetengine.initials.extents.IC(([1], planetengine.shapes.trapezoid()))}
    }

In [4]:
system = modelscripts.MS98X_systemscript.build(res = 64, f = 0.5, tau = 1e5)

In [5]:
planetengine.initials.apply(initial, system)

In [6]:
system.solve()

In [ ]:
planetengine.utilities.quickShow(system.temperatureField)

In [ ]:
# Example of how to build a 'scalar integral'
myint = planetengine.newstats.StandardIntegral(
    system.viscosityFn * system.velocityField,
    comp = 'mag',
    gradient = 'rad',
    surface = 'outer'
    )

In [ ]:
obsVars = {
    'temperature': system.temperatureField,
    'velocity': system.velocityField,
    'viscosity': system.viscosityFn,
    'stress': system.viscosityFn * system.velocityField
    }

In [ ]:
statsDict = {}

for varName, var in sorted(obsVars.items()):

    pevar = planetengine.standardise(var)
    var = pevar.var

    standardIntegralSuite = {
        'surface': ['volume', 'inner', 'outer'],
        'comp': ['mag', 'ang', 'rad'],
        'gradient': [None, 'ang', 'rad']
        }

    for inputDict in planetengine.utilities.suite_list(standardIntegralSuite):
        anVar = planetengine.newstats.StandardIntegral(
            var,
            **inputDict
            )
        statsDict[varName + '_' + anVar.opTag] = anVar

In [ ]:
statsDict

In [ ]:
resultsDict = {key: integral() for key, integral in sorted(statsDict.items())}

In [ ]:
resultsDict

In [ ]:
- resultsDict['temperature_radGrad_outerInt'] / resultsDict['temperature_innerInt']

In [ ]:
for i in range(1000):
    system.iterate()

In [ ]:
planetengine.quickShow(system.temperatureField, system.velocityField, system.materialVar)

In [ ]:
resultsDict = {key: integral() for key, integral in sorted(statsDict.items())}

In [ ]:
- resultsDict['temperature_radGrad_outerInt'] / resultsDict['temperature_innerInt']

In [ ]:
from underworld import function as fn

In [ ]:
planetengine.quickShow(fn.math.dot(system.mesh.unitvec_theta_Fn, system.viscosityFn * system.velocityField))